## Дообучение Saiga-Mistral, квантизация и инференс с помощью llama-cpp

В репозитории реализован код для дообучения русскоязычной LLM [Saiga mistral](https://huggingface.co/IlyaGusev/saiga_mistral_7b_lora), а также её квантизация и запуск с помощью llama-cpp. Попытался сделать код максимально гибким и воспроизводимым.  
Предполагается запуск на GPU. Может запускаться на multi-gpu без доп. модификаций.  
При создании ноутбука опирался на эту [статью](https://habr.com/ru/articles/776872/) на Хабре, задекорировал и актуализировал некоторые моменты

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import PeftModel, PeftConfig, AutoPeftModelForCausalLM
from datasets import load_dataset
import transformers
import torch
import time
import os

/home/ubuntu/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

st_time = time.time()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant=False,
)


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit = True,
    torch_dtype=torch.float16,
    device_map="auto",

)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    is_trainable = True,
    quantization=bnb_config
)

model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

print(generation_config)
print(f'Загрузка модели заняла {round(time.time() - st_time, 2)} секунд')

Loading checkpoint shards:   0%|                                                                                       | 0/2 [00:00<?, ?it/s]/home/ubuntu/miniconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.23s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9
}

Загрузка модели заняла 17.77 секунд


In [3]:
model.print_trainable_parameters()

trainable params: 16,777,216 || all params: 6,755,192,832 || trainable%: 0.24836028248556738


In [4]:
!nvidia-smi

Thu Feb 15 20:10:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10                     On  | 00000000:00:10.0 Off |                    0 |
|  0%   37C    P0              54W / 150W |   7804MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Загрузка датасета

Датасет для дообучения должен быть в формате json и иметь формат ```[{"system": str, "user": str, "bot": str}, ... ]```, где system - системное сообщение для модели (например, у Сайги в use-example это "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."), user - это промпт пользователя, bot - ответ модели.

In [5]:
TRAIN_PATH = "train.json"
VALID_PATH = "val.json"

data = load_dataset(
    "json", 
    data_files={
                'train': TRAIN_PATH,
                'validation': VALID_PATH
    }
)
data["train"] = data["train"].shuffle() # for train data shuffling, optional

Extracting data files: 100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1796.28it/s]
Generating train split: 100 examples [00:00, 6027.77 examples/s]
Generating validation split: 100 examples [00:00, 7337.19 examples/s]


## Предобработка датасета

In [6]:
CUTOFF_LEN = 2500 # до какого токена будет обрезать текст


def generate_prompt(data_point):
    prompt = f"""<s>system
{data_point['system']}</s><s>user
{data_point['user']}</s><s>bot
{data_point['bot']}[</s>"""
    return prompt
 
    
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
    result["labels"] = result["input_ids"].copy()
    return result


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt


In [7]:
train_data = (
    data["train"].map(generate_and_tokenize_prompt)
)

val_data = (
    data["validation"].map(generate_and_tokenize_prompt)
)

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1108.04 examples/s]


## Обучение модели

In [8]:
BATCH_SIZE = 6
MICRO_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_EPOCHS = 5
OUTPUT_DIR = "finetuned_model"

training_arguments = transformers.TrainingArguments(
            per_device_train_batch_size=MICRO_BATCH_SIZE,
            per_device_eval_batch_size=MICRO_BATCH_SIZE,
            prediction_loss_only=True,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
            num_train_epochs=TRAIN_EPOCHS,
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_steps=25000,
            optim="adamw_torch",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            output_dir=OUTPUT_DIR,
            load_best_model_at_end=True,
            report_to=None,
            overwrite_output_dir=True,
)

In [9]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [10]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model = torch.compile(model)
trainer.train()
model.save_pretrained(OUTPUT_DIR)

wandb: Currently logged in as: glebbondarchuk. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
0,No log,0.536956
1,No log,0.424719
3,No log,0.416695
4,No log,0.420964


## Квантизация модели

Для начала склонируем репозитории с библиотеками rulm и llama-cpp для конкатенации обученного адаптера и квантизации.

In [11]:
!git clone https://github.com/IlyaGusev/rulm.git
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'rulm'...
remote: Enumerating objects: 3164, done.
remote: Counting objects: 100% (634/634), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 3164 (delta 593), reused 579 (delta 565), pack-reused 2530
Receiving objects: 100% (3164/3164), 1.19 MiB | 5.69 MiB/s, done.
Resolving deltas: 100% (2124/2124), done.
Cloning into 'llama.cpp'...
remote: Enumerating objects: 18440, done.
remote: Counting objects: 100% (4727/4727), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 18440 (delta 4592), reused 4514 (delta 4476), pack-reused 13713
Receiving objects: 100% (18440/18440), 21.33 MiB | 4.59 MiB/s, done.
Resolving deltas: 100% (12888/12888), done.


### Склеим модель и обученный адаптер

In [12]:
from rulm.self_instruct.src.tools import convert_to_native

In [13]:
PATH_TO_CHECKPOINT = "finetuned_model/checkpoint-66" # путь до чекпоинта адаптера, который хотим приклеить
MERGED_MODEL_PATH = "merged_model.pt"

convert_to_native.convert_to_native(PATH_TO_CHECKPOINT, MERGED_MODEL_PATH, 
                                    device="cuda", enable_offloading=True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 291/291 [00:02<00:00, 110.29it/s]


Saving state_dict...


### Конвертируем склеенную модель в 16-битный формат GGUF для запуска с помощью llama-cpp

In [14]:
# сперва сохраним токенайзер в папку, где лежит лучший чекпоинт

tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga2_7b_lora", use_fast=False)
tokenizer.save_pretrained(PATH_TO_CHECKPOINT)

('finetuned_model/checkpoint-66/tokenizer_config.json',
 'finetuned_model/checkpoint-66/special_tokens_map.json',
 'finetuned_model/checkpoint-66/tokenizer.model',
 'finetuned_model/checkpoint-66/added_tokens.json')

Обязательная строчка, надо откатить версию llama-cpp, т.к. на последней квантизация почему-то не работает.

In [15]:
%cd llama.cpp
!git checkout 64e64aa

/home/ubuntu/jupyter-home-dir/test/llama.cpp
Note: switching to '64e64aa'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 64e64aa2 ggml : restore abort() in GGML_ASSERT (#4242)


In [16]:
OUTPUT_PATH = "../model-f16.gguf"

In [ ]:
!python convert.py {os.path.join("..", MERGED_MODEL_PATH)} --vocab-dir {os.path.join("..", PATH_TO_CHECKPOINT)} --outfile {OUTPUT_PATH} --outtype f16 --ctx 4096

### Квантуем моедль в 4 бита и 8 бит

In [20]:
!make quantize

I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -Wdouble-promotion -pthread -march=native -mtune=native 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -Wno-array-bounds -Wno-format-truncation -Wextra-semi -march=native -mtune=native 
I NVCCFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread    -Wno-pedantic -Xcompiler "-Wno-array-bounds -Wno-format-truncation -Wextra-semi -march=native -mt

In [21]:
QUANT_MODEL = "../model-q4_0.gguf"
QUANTIZATION_TYPE = "q4_0" # "q4_0" или "q4_1"

In [ ]:
! ./quantize {OUTPUT_PATH} {QUANT_MODEL} {QUANTIZATION_TYPE}

### Запуск скомпилированной версии на GPU с помощью llama-cpp

Переустановим llama-cpp на последнюю версию. Параметры, которые идут перед установкой, обязательны для запуска на GPU.

In [23]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 89.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 229.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 225.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 235.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 78.4 MB/s eta 0:00:0000:0100:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.43-cp311-cp311-manylinux_2_35_x86_64.whl size=20897076 sha256=94cf0559b46f9503629673d207694c1e6d3b973dd2b48d76dbbefd9e0e692b16
  Stored in directory: /tmp/pip-ephem-wheel-cache-nr7owji9/wheels/2a/9d/08/b558eac0caff83868235db10ed76b5b1d2ec06276cdd26dc5d
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
  

### Использование модели в питоновском коде

In [24]:
from llama_cpp import Llama

In [26]:
llm = Llama(model_path="../model-q4_0.gguf", n_gpu_layers=128, n_ctx=2048)

llama_model_loader: loaded meta data with 16 key-value pairs and 291 tensors from ../model-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 

In [27]:
!nvidia-smi

Thu Feb 15 20:28:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10                     On  | 00000000:00:10.0 Off |                    0 |
|  0%   37C    P0              55W / 150W |  10026MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [28]:
prompt = f"""<s>system
    {'Any system prompt'}</s><s>user
    {'Any user prompt'}</s><s>bot"""

In [30]:
start_time = time.time()

output = llm(
      prompt, # Prompt
      max_tokens=2048,
      echo=False,
      temperature=0
)

print(time.time() - start_time)


llama_print_timings:        load time =    7855.42 ms
llama_print_timings:      sample time =      15.88 ms /    47 runs   (    0.34 ms per token,  2959.88 tokens per second)
llama_print_timings: prompt eval time =    7855.31 ms /    19 tokens (  413.44 ms per token,     2.42 tokens per second)
llama_print_timings:        eval time =     574.44 ms /    46 runs   (   12.49 ms per token,    80.08 tokens per second)
llama_print_timings:       total time =    8608.36 ms /    65 tokens


8.61396336555481


In [ ]:
print(output["choices"][0]["text"][:-1])